In [10]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50

In [2]:
folder_path = r"C:\Users\shrey\Downloads\images_combined"
csv_path = r"C:\Users\shrey\Downloads\data_combined_new.csv"

In [3]:
# Load the CSV file
data = pd.read_csv(csv_path)

# Split the data into features (image names) and labels (beauty scores)
image_names = data['filename'].values
beauty_scores = data['score'].values

# Normalize the beauty scores
beauty_scores = beauty_scores / 5.0  

In [5]:
# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2, rotation_range=20, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)


# Create the training and validation generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=folder_path,
    x_col='filename',
    y_col='score',
    subset='training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other'
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=folder_path,
    x_col='filename',
    y_col='score',
    subset='validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other'
)

# Create the test generator
test_generator = test_datagen.flow_from_dataframe(
    dataframe=data,
    directory=folder_path,
    x_col='filename',
    y_col='score',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other'
)

Found 7932 validated image filenames.


C:\Users\shrey\anaconda3\envs\tensorf\Lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 77 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 1983 validated image filenames.
Found 9915 validated image filenames.


In [6]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [8]:
# Train the model
epochs = 10
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

C:\Users\shrey\AppData\Local\Temp\ipykernel_18416\4270206205.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10


248/248 [==============================] - 281s 1s/step - loss: 3.2727 - mae: 1.1641 - val_loss: 0.3935 - val_mae: 0.5156
Epoch 2/10
248/248 [==============================] - 215s 867ms/step - loss: 1.2753 - mae: 0.9016 - val_loss: 0.5325 - val_mae: 0.5642
Epoch 3/10
 18/248 [=>............................] - ETA: 2:58 - loss: 1.1283 - mae: 0.8578

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
test_loss, test_mae = model.evaluate_generator(test_generator, steps=len(test_generator))
print(f'Test Loss: {test_loss:.4f}')
print(f'Test MAE: {test_mae:.4f}')

In [11]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create the model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Compile the model
optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

model.summary()

94765736/94765736 [==============================] - 2s 0us/step


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 512)               51380736  
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
Total params: 74968961 (285.98 MB)
Trainable params: 51381249 (196.00 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [12]:
# Train the model
epochs = 30
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

C:\Users\shrey\AppData\Local\Temp\ipykernel_18416\1963811747.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/30
248/248 [==============================] - 558s 2s/step - loss: 47.1571 - mae: 1.7250 - val_loss: 0.4199 - val_mae: 0.5358
Epoch 2/30
248/248 [==============================] - 550s 2s/step - loss: 1.6308 - mae: 1.0168 - val_loss: 0.8594 - val_mae: 0.7388
Epoch 3/30
248/248 [==============================] - 538s 2s/step - loss: 1.5976 - mae: 1.0073 - val_loss: 0.7455 - val_mae: 0.6783
Epoch 4/30
248/248 [==============================] - 510s 2s/step - loss: 1.5223 - mae: 0.9904 - val_loss: 0.4978 - val_mae: 0.5414
Epoch 5/30
248/248 [==============================] - 542s 2s/step - loss: 1.4799 - mae: 0.9780 - val_loss: 0.3960 - val_mae: 0.5170
Epoch 6/30
248/248 [==============================] - 531s 2s/step - loss: 1.6345 - mae: 1.0093 - val_loss: 0.6073 - val_mae: 0.6063
Epoch 7/30
248/248 [==============================] - 532s 2s/step - loss: 1.4428 - mae: 0.9586 - val_loss: 0.4708 - val_mae: 0.5288
Epoch 8/30
248/248 [==============================] - 534s 2s/step -

In [13]:
# Evaluate the model on the test set
test_loss, test_mae = model.evaluate_generator(test_generator, steps=len(test_generator))
print(f'Test Loss: {test_loss:.4f}')
print(f'Test MAE: {test_mae:.4f}')

C:\Users\shrey\AppData\Local\Temp\ipykernel_18416\1381322539.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_mae = model.evaluate_generator(test_generator, steps=len(test_generator))


Test Loss: 0.3544
Test MAE: 0.4759


In [16]:
model.save('beauty_detection_model_full.h5')

C:\Users\shrey\anaconda3\envs\tensorf\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
cd Downloads

C:\Users\shrey\Downloads
